### RAA_미국주요지수

    1. Web : https://finance.naver.com/world/

    2. 해외증시
        a. 나스닥 종합[symbol : NAS@IXIC]
        b. S&P500[symbol : SPI@SPX]
        c. 다우 산업[symbol : DJI@DJI]

    3. ftn : 
        a."https://finance.naver.com/world/worldDayListJson.nhn?symbol={}&fdtc=0&page={}" 해당 URL에 symbol 과 페이지 번호(1~) 번호가 커질 수록 오래된 데이터

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
import json
import urllib.request as req

In [ ]:
cate = [];date = [];clos = []; diff = [];Open = [];high = [];low = []

def ftn(symbol,page):
    data = {"Query String Parameters": {'symbol': symbol,
                                    'fdtc': '0',
                                    'page': page}}
    req_header = {
    "authority": "finance.naver.com",
    "method": "POST",
    "path": '/world/worldDayListJson.nhn?symbol={}&fdtc=0&page={}'.format(symbol,page),
    "scheme": 'https',
    "accept": '*/*',
    "accept-encoding": 'gzip, deflate, br',
    "accept-language": 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    "charset": 'utf-8',
    "content-length": '0',
    'content-type': 'application/x-www-form-urlencoded; charset=utf-8',
    "cookie": 'NNB=VPHDOERS5XSGA; summary_item_type=recent; nx_ssl=2; naver_stock_codeList=086790%7C064850%7C383800%7C015260%7C003090%7C005930%7C337160%7C365590%7C; BMR=s=1626659707198&r=https%3A%2F%2Fm.blog.naver.com%2Fmegafollow%2F221491355891&r2=https%3A%2F%2Fwww.google.com%2F; page_uid=hngOVlp0YiRssEMD7eNssssstZ4-002498; _naver_usersession_=NmwGlLfu6lffO2KxxqOQIA==; JSESSIONID=1955D20367346D47362D0F4E2E4DFAE4',
    'origin': 'https://finance.naver.com',
    "referer": 'https://finance.naver.com/world/sise.nhn?symbol={}'.format(symbol),
    "sec-ch-ua": 'Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91',
    'sec-ch-ua-mobile': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    tmp = json.loads(requests.post("https://finance.naver.com/world/worldDayListJson.nhn?symbol={}&fdtc=0&page={}".format(symbol,page),
             data = json.dumps(data),
             headers = req_header).text)
    for num in range(len(tmp)):
        cate.append(tmp[num]["symb"])
        date.append(tmp[num]["xymd"])
        clos.append(tmp[num]["clos"])
        diff.append(tmp[num]["diff"])
        Open.append(tmp[num]["open"])
        high.append(tmp[num]["high"])
        low.append(tmp[num]["low"])

In [ ]:
sym = ['NAS@IXIC',"SPI@SPX","DJI@DJI"]
for tmp_sym in sym:
    for num in range(1,494):
        ftn(tmp_sym,str(num))
        print(tmp_sym,num)

In [ ]:
World = pd.DataFrame({"종목":cate,
             "날짜":date,
             "종가":clos,
             "대비":diff,
             "시가":Open,
             "고가":high,
             "저가":low})
World = World.drop_duplicates()
World.Date = World.Date.astype(str).map(lambda x : pd.to_datetime("-".join([x[:4],x[4:6],x[6:]])))
World['종목'] = World.종목.map(lambda x:x.replace("@","_"))

In [ ]:
World = World.rename(columns={
    "종목" : "해외지수구분코드",
    "날짜" : "기준일자",
    "종가" : "종가지수",
    "대비" : "전일대비증감지수"
})

In [ ]:
World.loc[:,['기준일자','해외지수구분코드','종가지수','전일대비증감지수']].to_csv('RAA_미국주요지수.csv')